Copyright (c) 2025, Infineon Technologies AG, or an affiliate of Infineon Technologies AG. All rights reserved.

This software, associated documentation and materials ("Software") is owned by Infineon Technologies AG or one of its affiliates ("Infineon") and is protected by and subject to worldwide patent protection, worldwide copyright laws, and international treaty provisions. Therefore, you may use this Software only as provided in the license agreement accompanying the software package from which you obtained this Software. If no license agreement applies, then any use, reproduction, modification, translation, or compilation of this Software is prohibited without the express written permission of Infineon.

Disclaimer: UNLESS OTHERWISE EXPRESSLY AGREED WITH INFINEON, THIS SOFTWARE IS PROVIDED AS-IS, WITH NO WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING, BUT NOT LIMITED TO, ALL WARRANTIES OF NON-INFRINGEMENT OF THIRD-PARTY RIGHTS AND IMPLIED WARRANTIES SUCH AS WARRANTIES OF FITNESS FOR A SPECIFIC USE/PURPOSE OR MERCHANTABILITY. Infineon reserves the right to make changes to the Software without notice. You are responsible for properly designing, programming, and testing the functionality and safety of your intended application of the Software, as well as complying with any legal requirements related to its use. Infineon does not guarantee that the Software will be free from intrusion, data theft or loss, or other breaches ("Security Breaches"), and Infineon shall have no liability arising out of any Security Breaches. Unless otherwise explicitly approved by Infineon, the Software may not be used in any application where a failure of the Product or any consequences of the use thereof can reasonably be expected to result in personal injury.

# Setup

In [ ]:
import os
import sys
from pathlib import Path

# Add current directory to path so we can import our modules
sys.path.append("/home/ubuntu")

from model_converter import ModelConverter
from config import *

print("🚀 Neural Network Conversion Tools - Debug Mode")
print("=" * 50)
print(f"Workspace: /home/ubuntu/workspace")
print(f"Tools available:")
print(f"  - onnx2c: {ONNX2C}")
print(f"  - testgen: {TESTGEN_TC3}")
print(f"  - testgen: {TESTGEN_TC4}")
print(f"  - gcc: {TC_GCC}")
print(f"  - qemu: {QEMU}")

# Configuration

- Modify these as needed

In [ ]:
MODEL_PATH = "/home/ubuntu/workspace/"  # Where your model.onnx is located
TARGET = "TC4"  # TC3, TC4, or PPU
OUTPUT_DIR = "/home/ubuntu/out/"

# Testgen parameters (adjustable)
RTOL = 0.01
ATOL = 0.01
RTOL_LIMIT = 0.1
ATOL_LIMIT = 0.1
TEST_DATA_SET = 0

# Create directories
os.makedirs(MODEL_PATH, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"✅ Configuration set:")
print(f"   Target: {TARGET}")
print(f"   Model path: {MODEL_PATH}")
print(f"   Output: {OUTPUT_DIR}")
print(f"   Tolerances: rtol={RTOL}, atol={ATOL}")

# Check Model

- Check if model exists

In [ ]:
model_files = list(Path(MODEL_PATH).glob("**/model.onnx"))
if model_files:
    print(f"✅ Found model: {model_files[0]}")

    # Show model info
    !ls -la {model_files[0]}
else:
    print("❌ No model.onnx found!")
    print("Please upload your model to the workspace/model directory")
    print("Current workspace contents:")
    !ls -la /home/ubuntu/workspace/

# Initialize Converter
- converter with debug output

In [ ]:
try:
    converter = ModelConverter(MODEL_PATH, TARGET)
    converter.rtol = RTOL
    converter.atol = ATOL
    converter.rtol_limit = RTOL_LIMIT
    converter.atol_limit = ATOL_LIMIT
    converter.test_data_set = TEST_DATA_SET

    print("✅ ModelConverter initialized successfully")
except Exception as e:
    print(f"❌ Failed to initialize converter: {e}")

# Conversion Step 1: Generate C Code

## Model only

In [ ]:
print("🔄 Step 1: Generating C code...")
try:
    converter.generate_c_file()

    if os.path.exists(converter.c_file):
        print(f"✅ C code generated: {converter.c_file}")
        print(f"File size: {os.path.getsize(converter.c_file)} bytes")

        # Show first few lines
        with open(converter.c_file, "r") as f:
            lines = f.readlines()[:20]
            print("\n📄 First 10 lines of generated C code:")
            print("".join(lines))
    else:
        print("❌ C code generation failed")

except Exception as e:
    print(f"❌ Error in C code generation: {e}")

## Testgen code generation

- The file that is going to be compiled

In [ ]:
print("🔄 Step 2: Generating testgen code...")
try:
    converter.generate_testgen_file()

    if os.path.exists(converter.testgen_file):
        print(f"✅ Testgen code generated: {converter.testgen_file}")
        print(f"File size: {os.path.getsize(converter.testgen_file)} bytes")

        # Show compilation info
        with open(converter.testgen_file, "r") as f:
            lines = f.readlines()[:20]
            print("\n📄 First 20 lines of generated C code:")
            print("".join(lines))

    else:
        print("❌ Testgen code generation failed")

except Exception as e:
    print(f"❌ Error in testgen generation: {e}")

# Step 3: Compile

In [ ]:
print("🔄 Step 3: Compiling...")
try:
    converter.compile_model()

    if os.path.exists(converter.elf_file):
        print(f"✅ Compilation successful: {converter.elf_file}")
        print(f"File size: {os.path.getsize(converter.elf_file)} bytes")

    else:
        print("❌ Compilation failed")

except Exception as e:
    print(f"❌ Error in compilation: {e}")

# Step 4: Simulate

In [ ]:
print("🔄 Step 4: Running simulation...")
try:
    converter.benchmark()

    # Check results
    if os.path.exists("QEMU.log"):
        print("✅ Simulation completed")
        print("\n📊 Simulation results:")
        with open("QEMU.log", "r") as f:
            results = f.read()
            print(results)

    else:
        print("❌ Simulation failed - no log file generated")

except Exception as e:
    print(f"❌ Error in simulation: {e}")

# Results Summary

In [ ]:
print("📋 Conversion Summary")
print("=" * 30)

files_to_check = [
    (converter.c_file, "C Code"),
    (converter.testgen_file, "Testgen Code"),
    (converter.elf_file, "Compiled ELF"),
    ("QEMU.log", "Simulation Results"),
]

for file_path, description in files_to_check:
    if os.path.exists(file_path):
        size = os.path.getsize(file_path)
        print(f"✅ {description}: {file_path} ({size} bytes)")
    else:
        print(f"❌ {description}: Not found")

print(f"✅ All results copied to {OUTPUT_DIR}")
!ls -la {OUTPUT_DIR}